# Medical Insurance (Medicare) Anomaly Detection Project As for Epsilon Graduation Project 

## EDA Stage

### 1- Data Reading & Understanding

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Healthcare Providers.csv')

In [ ]:
pd.set_option('display.max_columns' ,None) #to see all columns to understand them better
df

### 2- Report Generation

In [ ]:
print(' '*10,'Data Info:') # to Know general information about the dataset
df.info()
print('-'*100)
print(' '*10,'Columns and Rows count:') #to get summary of the dataset
print(df.shape)
print('-'*100)
print(' '*10,'Unique values of data in each column:')
print(df.nunique()) #to see unique values in each column to understand them better
print('-'*100)
print(' '*10,'Missing values in each column:')
print(df.isnull().sum()) #to see missing values in each column to understand them better 
print('-'*100)
print(' '*10,'Check for duplicates:')
print(df.duplicated().sum()) #to check for duplicates
print('-'*100)

### 3- Data Transformation 

In [ ]:
# Droping Unnecessary Columns for Analysis Process
df = df.drop(['Gender of the Provider','index','National Provider Identifier','Last Name/Organization Name of the Provider','First Name of the Provider','Middle Initial of the Provider','Credentials of the Provider','Street Address 2 of the Provider','Street Address 1 of the Provider','Zip Code of the Provider','Country Code of the Provider','Medicare Participation Indicator','HCPCS Description'
],axis = 1)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
columns_to_convert = [
    'Number of Services',
    'Number of Medicare Beneficiaries',
    'Number of Distinct Medicare Beneficiary/Per Day Services',
    'Average Medicare Allowed Amount',
    'Average Submitted Charge Amount',
    'Average Medicare Standardized Amount',
    'Average Medicare Payment Amount'
]

# Convert columns to numeric, coercing errors (non-numeric values become NaN)
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')


In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
import plotly.express as px
for column in columns_to_convert:
    fig = px.histogram(df, x=column)
    fig.show()

In [ ]:
columns_to_convert = [
    'Number of Services',
    'Number of Medicare Beneficiaries',
    'Number of Distinct Medicare Beneficiary/Per Day Services',
    'Average Medicare Allowed Amount',
    'Average Submitted Charge Amount',
    'Average Medicare Standardized Amount',
    'Average Medicare Payment Amount'
]

# Find and replace with NaN 
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Convert the columns to float
df[columns_to_convert] = df[columns_to_convert].astype(float)


In [ ]:
df.isna().sum()

In [ ]:
print(f"Original shape: {df.shape}")

# Remove rows with any NaN values in the specified columns
df = df.dropna(subset=columns_to_convert)

# Print the shape before and after to see how many rows were removed
print(f"Shape after removing rows with NaNs: {df.shape}")

# Check if there are any NaNs left in these columns
print("\nNaN counts after removal:")
print(df[columns_to_convert].isna().sum())

# Optionally, reset the index if needed
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
import plotly.express as px
for column in columns_to_convert:
    fig = px.histogram(df, x=column)
    fig.show()

In [ ]:
#Save the cleaned dataset to a new CSV file
df.to_csv('Healthcare_Providers_cleaned.csv', index=False)

### 4- Analysis & Pattern Recognition

#### 1- Categorical Features Analysis

In [ ]:
# mapping columns of binary features
label_mapping = {'I': 'Individual','O': 'Organization'}
df['Entity Type of the Provider'] = df['Entity Type of the Provider'].map(label_mapping)
place_of_service_mapping = {'F': 'Facility','O': 'Non-Facility'}
df['Place of Service'] = df['Place of Service'].map(place_of_service_mapping)
hcpcs_drug_indicator_mapping = {'Y': 'Drug Involved','N': 'No Drug'}
df['HCPCS Drug Indicator'] = df['HCPCS Drug Indicator'].map(hcpcs_drug_indicator_mapping)


binary_columns = ['Entity Type of the Provider','Place of Service','HCPCS Drug Indicator']
for column in binary_columns:
    fig = px.pie(df, names=column)
    fig.update_layout(title=f'Distribution of {column}')
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=12)
    fig.show()

In [ ]:
state_mapping = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia',
    'AS': 'American Samoa', 'GU': 'Guam', 'MP': 'Northern Mariana Islands', 'PR': 'Puerto Rico',
    'VI': 'Virgin Islands', 'UM': 'U.S. Minor Outlying Islands'
}
df['State Code of the Provider'] = df['State Code of the Provider'].map(state_mapping)
multiple_columns = ['HCPCS Code','City of the Provider','State Code of the Provider','Provider Type']
for column in multiple_columns:
    top_ten = df.groupby(column)['Number of Services'].sum().nlargest(10).reset_index()
    fig = px.bar(top_ten, x=column, y='Number of Services', color=column)
    fig.update_layout(title=f'Top 10 {column} by Number of Services')
    fig.show()

#### 2- Numerical Features Analysis

In [ ]:
df.describe()

In [ ]:
# Calculate the correlation matrix
corr = df.corr()

# Create the heatmap
fig = px.imshow(
    corr,
    text_auto=True,
    labels=dict(color="Correlation"),
    x=corr.columns.tolist(),
    y=corr.index.tolist()
)

# Remove axis labels
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

# Add hover text
fig.update_traces(
    hovertemplate="X: %{x}<br>Y: %{y}<br>Correlation: %{z:.2f}<extra></extra>"
)

base_width = 800
base_height = 600

fig.update_layout(
    width=base_width * 1.3,
    height=base_height * 1.3,  # Added comma here
    title={
        'text': "Correlation Heatmap",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

# Show the plot
fig.show()

In [ ]:
#box plot in numerical columns to see Anomalies and outliers
for column in columns_to_convert:
    fig = px.box(df, y=column)
    fig.show()